[あらゆる数独パズルを解く](http://www.aoky.net/articles/peter_norvig/sudoku.htm)
- http://www.aoky.net/articles/peter_norvig/code/easy50.txt
- http://magictour.free.fr/top95
- https://projecteuler.net/index.php?section=problems&id=96

In [76]:
def cross(A, B):
    "Aの要素とBの要素の外積。"
    return [a+b for a in A for b in B]
 
digits   = '123456789'
rows     = 'ABCDEFGHI'
cols     = digits
squares  = cross(rows, cols)
unitlist = ([cross(rows, c) for c in cols] +
            [cross(r, cols) for r in rows] +
            [cross(rs, cs) for rs in ('ABC','DEF','GHI') for cs in ('123','456','789')])
units = dict((s, [u for u in unitlist if s in u]) 
             for s in squares)
peers = dict((s, set(sum(units[s],[]))-set([s]))
             for s in squares)

In [84]:
n=9

In [301]:
idxs=np.arange(n**2)

In [303]:
grid=np.arange(n**2).reshape(n,n)

In [304]:
block=np.stack([grid[i*3:(i+1)*3,j*3:(j+1)*3].flatten() for i in range(3) for j in range(3)])

In [333]:
unitlist=np.concatenate([grid, grid.T,block])

In [337]:
def get_units(s):
    return unitlist[np.isin(unitlist,s).sum(-1).astype(bool)]

In [341]:
units=np.array([get_units(s) for s in range(n**2)])

In [350]:
def get_peers(s):
    a=np.unique(get_units(s))
    idx=a!=s
    return a[idx].tolist()

In [351]:
peers=np.array([get_peers(s) for s in idxs])

In [544]:
a="4.....8.5.3..........7......2.....6.....8.4......1.......6.3.7.5..2.....1.4......"
b= '003020600900305001001806400008102900700000008006708200002609500800203009005010300'

In [548]:
def myint(x):
    if x==".":
        return -1
    else:
        return int(x)-1
    
def grid_values(grid):
    "テキスト形式gridを辞書{square: char}に変換する。空のマスは'0'か'.'とする。"
    chars = [myint(c) for c in grid if c in (digits+1).astype(str) or c in '0.']
    assert len(chars) == 81
    return np.array(chars)

In [550]:
digits = np.arange(n)
def parse_grid(grid):
    """テキスト形式gridを可能な値の辞書{square: digits}に変換する。ただし
    矛盾が見つかった場合にはFalseを返す。"""
    ## 最初それぞれのマスは何の数字でもありうる。それからgridより値を割り当てる。
    values=np.full((n**2,n),True)
    for s,d in enumerate(grid_values(grid)):
        if d in digits and assign(values, s, d) is False:
            print(s,d)
            return False ## (マスsにdを割り当てられなければ失敗) 
    return values

In [551]:
def assign(values, s, d):
    """values[s]からd以外のすべての値を取り除き、伝播する。
   valuesを返す。ただし矛盾が見つかった場合はFalseを返す。"""
    other_values=values.copy()
    other_values[s,d]=False
    
    res=True
    for d2 in digits[other_values[s]]:
        res = eliminate(values, s, d2) is not False and res
    
    if res:
        return values
    else:
        return False

In [552]:
def eliminate(values, s, d):
    """ values[s]からdを取り除く。値か場所が1つになったら伝播する。
   valuesを返す。ただし、矛盾が見つかったときにはFalseを返す。"""
    if not values[s,d]:
        return values ## すでに取り除かれている
    values[s,d] = False
    ## (1) マスs が1つの値d2まで絞られたなら、ピアからd2を取り除く。
    if values[s].sum() == 0:
        return False ## 矛盾 最後の値が取り除かれた
    elif values[s].sum() == 1:
        # d2をピアから取り除けない場合矛盾
        d2 = int(digits[values[s]])
        for s2 in peers[s]:
            if eliminate(values, s2, d2) is False:
                return False
        
    ## (2) ユニットuで値dを置きうる場所が1カ所だけになったなら、dをその場所に入れる。
    for u in units[s]:
        dplaces = u[values[u][:,d]]
        
        if len(dplaces) == 0:
            return False ## 矛盾 値を置ける場所がない
        elif len(dplaces) == 1:
            # ユニットの中でdを置けるところが1カ所しかないので、そこに置く
            if assign(values, dplaces[0], d) is False:
                return False
    return values

In [519]:
values.sum(-1)

array([9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9,
       9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9,
       9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9,
       9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9])

In [557]:
def display(values):
    "valuesを2次元のテキスト形式で表示する。"
    width = 1+values.sum(-1).max()
    line = '+'.join(['-'*(width*3)]*3)
    
    def tostr(x):
        return "".join((digits[x]+1).astype(str)).center(width)
    
    x=np.apply_along_axis(tostr, -1, values[grid])
    for r in range(n):
        print(*list(x[r,c]+('|' if c == 2 or c == 5 else '') for c in range(n)))
        if r == 2 or r==5: print(line)

In [611]:
def solve(grid): return search(parse_grid(grid))
 
    
def search(values):
    "深さ優先探索と制約伝播を使い、すべての可能なvaluesを試す。"
    if values is False:
        return False ## 前の時点で失敗している
    if (values.sum(-1)==1).prod():
        return values ## 解けた!
    
    ## 取り得る値の個数が最小である未確定のマスsを選ぶ
    x=values.sum(-1)
    x[x==1] *= 10
    s=x.argmin()
    
    return some(search(assign(values.copy(), s, d)) for d in digits[values[s]])
 
def some(seq):
    "seqの要素からtrueであるものをどれか返す。"
    for e in seq:
        if e is not False: return e
    return False